### Import Modules

In [1]:
import fiona
from shapely.geometry import shape, Point
from shapely import speedups
speedups.enable()
import datetime
import pandas

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


### Set paths

In [2]:
points = "/users/danielcorcoran/desktop/test_coordinates_10K.csv"
shapefile = "/users/danielcorcoran/desktop/github_repos/python_nb_data_spatial/data/LGA_ABS_16/LGA_2016_AUST.shp"

### Import data and preview

In [3]:
#data = pandas.read_csv("/users/danielcorcoran/desktop/test_coordinates_10K.csv")

In [4]:
import numpy
x  = [numpy.random.uniform(140,150) for n in range(200000)]
y  = [numpy.random.uniform(-32,-40) for n in range(200000)]
data = pandas.concat([pandas.Series(x), pandas.Series(y)], axis = 1)
data.columns = ['longitude', 'latitude']

In [5]:
print('Checking data shape:\n{}'.format(data.shape))

Checking data shape:
(200000, 2)


In [6]:
print('Checking data headers:\n{}'.format(list(data.columns)))

Checking data headers:
['longitude', 'latitude']


In [7]:
long_name = 'longitude'
lat_name = 'latitude'

### Create list of polygons containing Victoria only from LGA shapefile

In [8]:
all_polygons = [polygon for polygon in fiona.open(shapefile)]

In [9]:
vic_polygons = []

for polygon in all_polygons:
    if polygon['properties']['STE_NAME16'] == 'Victoria':
        vic_polygons.append(polygon)

### Build spatial tree

In [10]:
from rtree import index
rtree_index = index.Index()

In [11]:
for position, polygon in enumerate(vic_polygons):
    try:
        rtree_index.insert(position, shape(polygon['geometry']).bounds)
    except:
        print("Error at position", position)

Error at position 80
Error at position 81


### Process  (with apply)

In [12]:
# Create function to return the properties for a shapefile if a match is found in rtree_index
def return_properties(point):
    for index in rtree_index.intersection(point.coords[0]):
        if point.within(shape(vic_polygons[index]['geometry'])):
            return vic_polygons[index]['properties']

In [13]:
# Create new features with point objects using long and lat features
data['point'] = (list(zip(data[long_name], data[lat_name])))
data["shapely_point"] = data['point'].apply(Point)
shapely_points = list(data['shapely_point'])

In [14]:
from multiprocessing import Pool
p = Pool(4)

In [ ]:
%%time

data['results'] = list(p.map(return_properties, shapely_points))

# Join original data with results series split into a new dataframe
# (one column for each property)
data = pandas.concat([data, data['results'].apply(pandas.Series)], axis = 1)

# Drop irrelevant columns
data.drop(['shapely_point', 'results', 'point'], axis = 1, inplace = True)

In [ ]:
print(data.head())

In [ ]:
print('Points found {}. Total Points {}. % Points found {:.2%}'.format(data['LGA_CODE16'].notnull().sum(),
                                                                        data['LGA_CODE16'].shape[0],
                                (data['LGA_CODE16'].notnull().sum()/data['LGA_CODE16'].shape[0])))

### Export

In [ ]:
data.to_csv('/users/danielcorcoran/desktop/test_points.csv', index = False)